# Analyse des attaques DOS détectées
Ce notebook contient l'analyse des données collectées lors de la simulation et de la détection d'attaques par déni de service (DOS). Nous allons répondre à deux questions :

1. Où se trouvent les adresses IP suspectes ?
2. Quelle est l'attaque la plus courante ?


In [ ]:
import pandas as pd

# Charger les attaques détectées depuis le fichier CSV
df = pd.read_csv('detected_attacks.csv')

# Afficher les premières lignes du fichier pour vérifier les données
df.head()


In [ ]:
import requests

# Fonction pour obtenir la localisation d'une adresse IP
def get_ip_location(ip):
    try:
        response = requests.get(f"http://ip-api.com/json/{ip}")
        data = response.json()
        return data['country'], data['lat'], data['lon']
    except:
        return "Unknown", None, None

# Appliquer la fonction sur toutes les IP détectées
df[['Location', 'lat', 'lon']] = df['Source IP'].apply(lambda ip: pd.Series(get_ip_location(ip)))

# Afficher les résultats
df[['Source IP', 'Location', 'lat', 'lon']].head()


In [ ]:
import folium

# Créer une carte centrée autour de la latitude et de la longitude moyenne
map = folium.Map(location=[20, 0], zoom_start=2)

# Ajouter des marqueurs pour chaque IP détectée
for index, row in df.iterrows():
    if row['Location'] != "Unknown" and row['lat'] and row['lon']:
        folium.Marker([row['lat'], row['lon']], popup=row['Source IP']).add_to(map)

# Afficher la carte
map


In [ ]:
import matplotlib.pyplot as plt

# Compter les occurrences de chaque type d'attaque
attack_counts = df['Attack Type'].value_counts()

# Afficher les attaques les plus fréquentes
print(attack_counts)

# Créer un graphique des attaques les plus fréquentes
attack_counts.plot(kind='bar')
plt.title('Types d\'attaques les plus fréquentes')
plt.xlabel('Type d\'attaque')
plt.ylabel('Nombre d\'occurrences')
plt.show()


In [ ]:
# Analyse des pays les plus fréquents pour les IP suspectes
top_countries = df['Location'].value_counts().head(3).index.tolist()

# Analyse du type d'attaque le plus fréquent
most_common_attack = df['Attack Type'].value_counts().idxmax()

# Générer la conclusion dynamique
conclusion = f"""
## Conclusion dynamique

Nous avons localisé les adresses IP suspectes provenant principalement des pays suivants : **{', '.join(top_countries)}**.

L'attaque la plus courante détectée est de type **{most_common_attack}**.
"""

# Afficher la conclusion
from IPython.display import Markdown
Markdown(conclusion)


## Conclusion dynamique

Nous avons localisé les adresses IP suspectes provenant principalement des pays suivants : **France, USA, Canada**.

L'attaque la plus courante détectée est de type **syn_ack**.
